In [1]:
import pandas as pd
import numpy as np
from scipy import signal
from numpy import savetxt
import os

In [2]:
def creat_df(fle):
    df = pd.read_csv(path + fle, sep = ',')
    df = df.rename(columns = {'Column1':'Time', 'Column2':'Vm','ICaL':'ICaL','IK1':'IK1','Ikr':'Ikr','IKur':'IKur','INa':'INa'})
    df['Time'] = df['Time']-45000
    df = df.dropna()
    return df

In [3]:
def split_date(df):
    look_1 = df.loc[:24999,["Vm"]]
    look_2 = df.loc[25000:49000,["Vm"]]
    look_3 = df.loc[50000:74999,["Vm"]]
    look_4 = df.loc[75000:99999,["Vm"]]
    return look_1, look_2, look_3, look_4
    

In [4]:
def date_12(X_ds, Y_ds,look):
    index = look[look['Vm'] == look['Vm'].min()]
    prime = index.index.values[0]
    l_prime = prime + 12000

    x_ds = df[['ICaL','IK1','IKr','IKur','INa']].iloc[0]
    x_ds = x_ds.to_numpy()
    x_ds = x_ds.reshape((5,1))

    y_ds = df[['Vm']].iloc[prime:l_prime]
    y_ds = y_ds.to_numpy()
    

    if y_ds.shape == (12000,1):
               
        X_ds = np.append(X_ds,x_ds,axis = 0)
        Y_ds = np.append(Y_ds,y_ds,axis = 0)
    return X_ds, Y_ds

In [5]:
def downsampling(Y_new,i):
    
    y_new = signal.resample(Y_ds[i],5000)
    y_new = np.delete(y_new,slice(0,50),0)
    y_new = np.delete(y_new,np.s_[4900:4950],0)
    Y_new = np.append(Y_new,y_new)
    
    return Y_new
    



Data

In [6]:
path = 'DATE/'
all_files = os.listdir(path)

In [7]:
X_ds  = np.zeros([5,1])
Y_ds  = np.zeros([12000,1])

In [12]:
for fle in all_files:
    df = creat_df(fle)
    look_1, look_2, look_3, look_4 = split_date(df)
    X_ds, Y_ds = date_12(X_ds, Y_ds,look_1)
    X_ds, Y_ds = date_12(X_ds, Y_ds,look_2)
    X_ds, Y_ds = date_12(X_ds, Y_ds,look_3)
    X_ds, Y_ds = date_12(X_ds, Y_ds,look_4)

In [13]:
X_ds = X_ds.reshape((971,5))
Y_ds = Y_ds.reshape((971,12000))
X_ds = np.delete(X_ds,0,0)
Y_ds = np.delete(Y_ds,0,0)
print(X_ds.shape)
print(Y_ds.shape)

(970, 5)
(970, 12000)


Downsampling

In [14]:
Y_new = np.zeros([4900,1])
for i in range(970):
    Y_new = downsampling(Y_new,i)
Y_new = Y_new.reshape((971,4900))
Y_new = np.delete(Y_new,0,0)

In [15]:
savetxt('train_x.csv', X_ds , delimiter=',')
savetxt('train_y.csv', Y_new , delimiter=',')